# Projet Neo4j sur la base de données BROT

### Contexte 
Faire une mise en contexte

## Problématique 
A partir de ce jeu de données, les problématiques sont très nombreuses. Dans un premier temps, comme l'auteur a identifié des traits en lien avec la résillience des plantes au incendie, j'ai décidé de me concentrer sur l'étude de ces traits. 

## Adaptation aux incendies

In [15]:
from db_request import *

#### Identification des familles dont certaines espèces sont adaptées aux incendies

In [48]:
# PENSER A REMPLACER pd.unique PAR DISTINCT avec NEO4J !!

def allTaxons():
    rq = ''' match(tx:Taxon) return tx.family as family, tx.genus as genre, tx.species as espece'''
    return graph.run(rq).to_data_frame()

allTaxons=allTaxons()
allFamilies=pd.unique(allTaxons["family"])

print(f"Nombre de taxons : {len(allTaxons)}")
print(f"Nombre famille : {len(allFamilies)}")

Nombre de taxons : 2457
Nombre famille : 119


In [40]:
fireStimFlower = allTaxonWithTraitAndModality("FireStimFlower", "yes")
fireStimFlowerFamilies = pd.unique(fireStimFlower["famille"])
print(f"Nombre de plante possédant ce trait : {len(fireStimFlower)}")
print(f"Nombre de familles ayant au moins une espèce dont la floraison est stimulée pour les incendies : {len(fireStimFlowerFamilies)}")
print(fireStimFlowerFamilies)

Nombre de plante possédant ce trait : 24
Nombre de familles ayant au moins une espèce dont la floraison est stimulée pour les incendies : 8
['Asphodelaceae' 'Primulaceae' 'Asparagaceae' 'Amaryllidaceae' 'Liliaceae'
 'Dioscoreaceae' 'Araceae' 'Iridaceae']


In [47]:
percentageOfFireStimFlowerFamily = (len(fireStimFlowerFamilies)/len(allFamilies))*100
print(percentageOfFireStimFlowerFamily)

percentageOfFireStimFlowerTaxon = (len(fireStimFlower) / len(allTaxons)) * 100
print(percentageOfFireStimFlowerTaxon)

6.722689075630252
0.9768009768009768


Seulement 6% des familles possédent au moins une espèce ayant le trait **FireStimFlower**. 
Et moins d'1% des plantes voient leur floraison stimulée par le feu. 

#### Focus sur le trait FireStimFlower

D'après la documentation, ce trait indique que les plantes le possédant sont capables (voire nécessitent) de fleurir après des incendies. 
Si l'on regarde un peu plus en détail la documentation, il est indiqué que ce sont les plantes géophytes qui présentent le plus ce trait. Une plante géophyte est une plante qui possède un organe permettant de "passer la mauvaise saison" (wikipédia), tel qu'un bulbe ou bien un rhizome. On peut donc regarder la proportion de plantes géophytes dans celle présentant le trait **FireStimFlower**. 

Pour ce faire, pour chaque plante présentant le trait FireStimFlower, on teste si elle possède certaine modalité du type BudSource.

In [75]:
def modalityOfOneTraitForOneTaxon(taxon, trait):
    rq = '''match(tx:Taxon {taxon: $taxon})<-[:TRAIT_OF_TAXON]-(d:Data)-[:TYPE_OF_TRAIT]->(t:Trait {name: $trait}) return tx.taxon as taxon, d.data as trait'''
    res = graph.run(rq, taxon=taxon, trait=trait).to_data_frame()
    if(len(res) != 0):
        return pd.unique(res["trait"])
    return []

def hasOneOfThisTrait(traits, taxonTraits):
    for t in taxonTraits:
        if t in traits:
            return True
    return False

In [78]:
for el in fireStimFlower["taxon"]:
    print(el, modalityOfOneTraitForOneTaxon(el, "BudSource"))

Asphodelus ramosus ['storage organs']
Cyclamen hederifolium []
Muscari racemosum []
Urginea maritima ['storage organs' 'rhizomes or roots']
Narcissus triandrus []
Narcissus dubius []
Gagea peduncularis []
Dioscorea communis []
Acis nicaeensis ['storage organs']
Ornithogalum narbonense []
Arisarum vulgare ['storage organs']
Gladiolus illyricus ['storage organs']
Iris lutescens ['rhizomes']
Lapiedra martinezii ['storage organs']
Gladiolus illyricus ['storage organs']
Romulea tempskyana []
Ornithogalum sphaerocarpum []
Allium hirtovaginatum []
Allium sandrasicum []
Allium amethystinum []
Gagea graeca ['storage organs']
Leopoldia comosa ['storage organs']
Asparagus stipularis []
Allium stamineum []


Le trait **BudSource** renseigne "l'origine" des bourgeons (bud). 
On peut déjà constater que pour les plantes dont la floraison est stimulée par le feu et pour lesquelles on a l'information concernant le trait **BudSource**, seuls les organes de stockages (bulbes) et les rhizomes sont recensés. 

Ces plantes sont donc pour la plupart des géophytes. 

Nous pouvons aller plus loin dans l'étude de ces plantes présentant le trait **FireStimFlower** en regardant quel trait ils possèdent tous et voir s'il y a d'autres similarités. 